***EN ESTE NOTEBOOK IMPLEMENTAMOS UNA PARTE DE NUESTRO CHATBOT, EL USUARIO INTRODUCIRÁ UN MÚSCULO A TRABAJAR Y CON AYUDA DE UN MODELO SENTENCE SIMILARITY GENERAREMOS 5 EJERCICIOS***

In [1]:
from sentence_transformers import SentenceTransformer, util
import json

- Modelo utilizado : *'paraphrase-MiniLM-L6-v2'*

In [2]:
# Inicializar un modelo
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


/home/ayala/miniconda3/envs/gym/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ayala/miniconda3/envs/gym/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


- Cargamos nuestro archivo *'datos_ejercicios.json'*

In [3]:
with open("../../json_data/datos_ejercicios_submuestreo.json", "r") as file:
    datos = json.load(file)

- Definimos una función para poder preprocesar las decripciones de los ejercicios

In [4]:
def clean_description(description):
    cleaned_description = description.lower()
    return cleaned_description

- Definimos la función *'find_similar_exercises'* que será la encargada de buscar similitudes entre las descripciones de los ejercicios, en este caso, el usuario nos dará una parte del cuerpo a entrenar, y filtraremos los ejercicios que pertenezcan a esa parte del cuerpo y una vez filtrados, extreaemos las descripciones de cada ejercicio y comparamos la similitud entre ellas

- Resumen función : 

    - Buscar ejercicios que trabajen la parte del cuerpo específica

    - Extraer las descripciones de los ejercicios

    - Codificar las descripciones de los ejercicios con .encode()
    
    - Funcion anidada para calcular la similitud coseno


In [5]:
def find_similar_exercises(body_part, top_n=5):

    buscar_ejercicios = [exercise for exercise in datos if exercise["BodyPart"].lower() == body_part.lower()]
    if not buscar_ejercicios:
        print(f"No se encontraron ejercicios para la parte del cuerpo '{body_part}' en el conjunto de datos.")
        return []

    descripcion_ejercicio = [exercise["Desc"] for exercise in buscar_ejercicios]
    exercise_embeddings = model.encode(descripcion_ejercicio, convert_to_tensor=True)
    
    def cosine_similarity(query_embedding):
        similarities = util.pytorch_cos_sim(query_embedding, exercise_embeddings)[0]
        similar_indices = similarities.argsort(descending=True)
        similar_exercises = []
        for i in similar_indices:
            title = buscar_ejercicios[i]["Title"]
            description = buscar_ejercicios[i]["Desc"]
            body_part = buscar_ejercicios[i]['BodyPart']
            similarity = similarities[i].item()
            if title not in [ex[0] for ex in similar_exercises]:
                similar_exercises.append((title, description, similarity,body_part))
            if len(similar_exercises) >= top_n:
                break
        return similar_exercises

    return cosine_similarity

- Proporcionamos un ejemplo de como sería el funcionamiento e imprimimos el título de cada ejercicio depende de la parte del cuerpo proporcionada por el usuario, además de su descripción y la parte del cuerpo que trabaja

In [6]:
# Ejemplo
body_part_input = input("Introduce la parte del cuerpo para la que quieres ejercicios: ")
devolver_ejercicios_similares = find_similar_exercises(body_part_input)

if devolver_ejercicios_similares:
    ejercicios_similares = devolver_ejercicios_similares(model.encode(clean_description("")))
    for title, description, similarity, body_part in ejercicios_similares:
        print(f"***Ejercicio***\nTítulo: {title}\nDescripción: {description}\nParte del cuerpo: {body_part}\n")

***Ejercicio***
Título: Medicine Ball Chest Pass
Descripción: The medicine ball pass is a simple partner-based exercise that involves throwing a weighted ball back-and-forth. It can be part of a dynamic warm-up for lifting or athletics, but can also work as active rest or as a component of circuit training.
Parte del cuerpo: Chest

***Ejercicio***
Título: Side kick-through push-up
Descripción: The side kick-through push-up is a full-body movement that begins in a bear crawl position and ends with a leg kicked out to the side, with a push-up performed between kick-throughs. It is commonly performed for reps or timed bodyweight circuits and animal-style flows, but can also work on its own or as part of a dynamic warm-up or mobility sequence.
Parte del cuerpo: Chest

***Ejercicio***
Título: Svend press
Descripción: The Svend press is an exercise where the chest is targeted by performing presses while squeezing two weight plates together. It is usually performed for relatively high reps wi